<a href="https://colab.research.google.com/github/JeffCostaF/address-validator-gemini/blob/main/Validador_de_Endere%C3%A7os_com_Gemini_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install -q google-generativeai gspread pandas geopy python-dotenv google-auth-httplib2 google-api-python-client oauth2client requests

In [36]:
# config.py
import os
from google.colab import userdata

# Configuração da API Key do Gemini
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

if not GEMINI_API_KEY:
    print("ALERTA: GEMINI_API_KEY não encontrada nos Secrets do Colab! O script não funcionará.")

# Configurações da Planilha Google
GOOGLE_SHEET_ID = "{ID_DA_PLANILHA_AQUI}"
GOOGLE_SHEET_CREDENTIALS_FILE = "credentials.json" # Lembre-se de fazer upload
SHEET_NAME_TO_PROCESS = "enderecos_com_problema"

# API de Consulta de CEP
VIACEP_BASE_URL = "https://viacep.com.br/ws/{cep}/json/"

# Modelo Gemini
GEMINI_MODEL_ID = "gemini-1.5-flash-latest" # Ou outro modelo adequado

# Nomes das colunas de ENTRADA na planilha
COL_LOGRADOURO_ORIGINAL = "Logradouro Original"
COL_NUMERO_ORIGINAL = "Número Original"
COL_COMPLEMENTO_ORIGINAL = "Complemento Original"
COL_BAIRRO_ORIGINAL = "Bairro Original"
COL_CIDADE_ORIGINAL = "Cidade Original"
COL_UF_ORIGINAL = "UF Original"
COL_CEP_ORIGINAL = "CEP Original"

# Nomes das colunas de SAÍDA que serão adicionadas/atualizadas
# Resultados da consulta ViaCEP
COL_CEP_CONSULTADO_LOGRADOURO = "Logradouro (ViaCEP)"
COL_CEP_CONSULTADO_BAIRRO = "Bairro (ViaCEP)"
COL_CEP_CONSULTADO_CIDADE = "Cidade (ViaCEP)"
COL_CEP_CONSULTADO_UF = "UF (ViaCEP)"
COL_STATUS_CONSULTA_CEP = "Status Consulta CEP" # Ex: "Encontrado", "Não Encontrado", "Inválido"

# Resultados da validação e correção pelo Gemini
COL_LOGRADOURO_FINAL = "Logradouro Final Corrigido"
COL_NUMERO_FINAL = "Número Final Corrigido"
COL_COMPLEMENTO_FINAL = "Complemento Final Corrigido"
COL_BAIRRO_FINAL = "Bairro Final Corrigido"
COL_CIDADE_FINAL = "Cidade Final Corrigida"
COL_UF_FINAL = "UF Final Corrigida"
COL_CEP_FINAL = "CEP Final Corrigido"
COL_ENDERECO_COMPLETO_FINAL = "Endereço Completo Final" # A coluna única que você pediu
COL_STATUS_VALIDACAO_FINAL = "Status Final da Validação" # O status claro
COL_OBSERVACAO_VALIDACAO = "Observações da Validação (Gemini)"
COL_JSON_GEMINI_FINAL = "JSON Retorno Gemini (Final)" # Para auditoria da resposta do Gemini

# Resultados da Geolocalização
COL_LATITUDE = "Latitude"
COL_LONGITUDE = "Longitude"
COL_LINK_Maps = "Link Google Maps"

# Lista de todas as colunas de resultado que o script pode criar/atualizar
ALL_RESULT_COLUMNS = [
    COL_CEP_CONSULTADO_LOGRADOURO, COL_CEP_CONSULTADO_BAIRRO, COL_CEP_CONSULTADO_CIDADE,
    COL_CEP_CONSULTADO_UF, COL_STATUS_CONSULTA_CEP,
    COL_LOGRADOURO_FINAL, COL_NUMERO_FINAL, COL_COMPLEMENTO_FINAL, COL_BAIRRO_FINAL,
    COL_CIDADE_FINAL, COL_UF_FINAL, COL_CEP_FINAL, COL_ENDERECO_COMPLETO_FINAL,
    COL_STATUS_VALIDACAO_FINAL, COL_OBSERVACAO_VALIDACAO, COL_JSON_GEMINI_FINAL,
    COL_LATITUDE, COL_LONGITUDE, COL_LINK_Maps
]

print("INFO: Configurações carregadas.")
if GEMINI_API_KEY:
    print(f"INFO: API Key do Gemini carregada. Modelo Gemini: {GEMINI_MODEL_ID}")
print(f"INFO: Planilha ID: {GOOGLE_SHEET_ID}, Aba: {SHEET_NAME_TO_PROCESS}")
print(f"INFO: API ViaCEP URL base: {VIACEP_BASE_URL.format(cep='exempl0')}")

INFO: Configurações carregadas.
INFO: API Key do Gemini carregada. Modelo Gemini: gemini-1.5-flash-latest
INFO: Planilha ID: 1zRX--zEYZTaIJIuX6oQObw2V9KGqpDgS242jxehPoL8, Aba: enderecos_com_problema
INFO: API ViaCEP URL base: https://viacep.com.br/ws/exempl0/json/


In [37]:
# cep_handler.py
import requests
import json
import re # Para limpar o CEP

# VIACEP_BASE_URL é definido na Célula 2 (config.py)

def clean_cep(cep_str: str) -> str:
    """Limpa o CEP, removendo caracteres não numéricos."""
    if not cep_str or not isinstance(cep_str, str):
        return ""
    return re.sub(r'\D', '', cep_str)

def get_address_from_viacep(cep: str) -> dict:
    """
    Consulta a API ViaCEP para obter dados de endereço a partir de um CEP.
    Retorna um dicionário com os dados do endereço e um status da consulta.
    """
    cleaned_cep = clean_cep(cep)
    if len(cleaned_cep) != 8:
        print(f"ALERTA (ViaCEP): CEP '{cep}' (limpo: '{cleaned_cep}') é inválido em formato.")
        return {
            "status_consulta_cep": "CEP com formato inválido",
            "logradouro": None, "bairro": None, "localidade": None, "uf": None, "erro": True
        }

    try:
        url = VIACEP_BASE_URL.format(cep=cleaned_cep)
        print(f"INFO (ViaCEP): Consultando CEP: {cleaned_cep} na URL: {url}")
        response = requests.get(url, timeout=5) # Timeout de 5 segundos
        response.raise_for_status() # Levanta exceção para erros HTTP (4xx ou 5xx)

        data = response.json()

        if data.get("erro"): # ViaCEP retorna {"erro": true} para CEPs não encontrados
            print(f"ALERTA (ViaCEP): CEP '{cleaned_cep}' não encontrado na base do ViaCEP.")
            return {
                "status_consulta_cep": "CEP não encontrado",
                "logradouro": None, "bairro": None, "localidade": None, "uf": None, "erro": True
            }

        print(f"INFO (ViaCEP): Endereço encontrado para CEP '{cleaned_cep}': {data.get('logradouro')}, {data.get('bairro')}, {data.get('localidade')}-{data.get('uf')}")
        return {
            "status_consulta_cep": "Encontrado",
            "logradouro": data.get("logradouro"),
            "bairro": data.get("bairro"),
            "localidade": data.get("localidade"), # 'localidade' é a cidade no ViaCEP
            "uf": data.get("uf"),
            "erro": False
        }

    except requests.exceptions.Timeout:
        print(f"ERRO (ViaCEP): Timeout ao consultar CEP '{cleaned_cep}'.")
        return {"status_consulta_cep": "Timeout na consulta", "erro": True}
    except requests.exceptions.RequestException as e:
        print(f"ERRO (ViaCEP): Erro na requisição para CEP '{cleaned_cep}': {e}")
        return {"status_consulta_cep": f"Erro na requisição: {e}", "erro": True}
    except json.JSONDecodeError:
        print(f"ERRO (ViaCEP): Erro ao decodificar JSON da resposta para CEP '{cleaned_cep}'.")
        return {"status_consulta_cep": "Erro na resposta JSON", "erro": True}

print("INFO: Handler para consulta de CEP (ViaCEP) definido.")

INFO: Handler para consulta de CEP (ViaCEP) definido.


In [45]:
# prompts.py

# Nomes de colunas (COL_..._ORIGINAL) são definidos na Célula 2 (config.py)
# e são usados aqui para referência na formatação do prompt que é enviado ao Gemini.

GEMINI_MULTI_FASE_VALIDATION_PROMPT = """
Você é um sistema avançado de inteligência artificial especializado na validação, correção e padronização de endereços brasileiros. Sua tarefa é analisar um endereço fornecido (original da planilha) e, quando disponível, um endereço de referência obtido através de uma consulta de CEP. Você deve retornar um objeto JSON estruturado com o endereço final corrigido e informações sobre o processo.

**FASES DA SUA ANÁLISE:**

**FASE 1: Padronização de Abreviações e Formatação Inicial**
   - Independente de qualquer outra informação, padronize TODOS os tipos de logradouro no endereço original da planilha (Ex: R., RUA, AV, Av., TR, Trv, etc., para Rua, Avenida, Travessa, Estrada, Praça, Alameda, Rodovia, etc.).
   - Formate o CEP original da planilha (se houver) para XXXXX-XXX.

**FASE 2: Análise Comparativa e Correção Inteligente**
   Você receberá:
   1. `endereco_planilha`: {endereco_planilha_str} (Este já deve estar com abreviações de logradouro parcialmente padronizadas por você na Fase 1)
      - Logradouro Original: {logradouro_planilha}
      - Número Original: {numero_planilha}
      - Complemento Original: {complemento_planilha}
      - Bairro Original: {bairro_planilha}
      - Cidade Original: {cidade_planilha}
      - UF Original: {uf_planilha}
      - CEP Original (formatado por você): {cep_planilha}

   2. `endereco_consulta_cep` (pode não existir se o CEP original for inválido ou não fornecido):
      - Status da Consulta CEP: {status_consulta_cep}
      - Logradouro do CEP: {logradouro_cep}
      - Bairro do CEP: {bairro_cep}
      - Cidade do CEP: {cidade_cep}
      - UF do CEP: {uf_cep}

   **Sua Tarefa Principal nesta Fase:**
   a. **Comparar** `endereco_planilha` com `endereco_consulta_cep`.
   b. **Identificar Consistências e Divergências.**
   c. **Correção e Decisão (Usar IA Avançada):**
      - **Priorize a informação do `endereco_consulta_cep` para logradouro, bairro, cidade e UF se o `status_consulta_cep` for "Encontrado" e o CEP da planilha parecer confiável.** Avalie se o logradouro da planilha, mesmo diferente, pode ser uma especificação mais detalhada.
      - **Use o `endereco_planilha` para obter o NÚMERO e COMPLEMENTO.** Padronize e corrija-os.
      - Se houver divergências significativas, analise qual fonte parece mais provável.
      - Se o `status_consulta_cep` indicar falha, foque em corrigir e padronizar o `endereco_planilha` usando seu conhecimento geral. Tente inferir o CEP correto.
      - **Padronize TODOS os campos de texto.**
      - O número do endereço deve ser mantido/corrigido. Se ausente, deixe como nulo ou vazio.
      - Complementos devem ser mantidos e limpos.
      - **Avalie o endereço como um TODO CONCATENADO** para garantir coerência final.

**FASE 3: Geração do Endereço Completo Final e Status**
   - Crie uma string única `endereco_completo_final` no formato: "Tipo Logradouro Nome Logradouro, Número, Complemento (se houver), Bairro - Cidade/UF, CEP: XXXXX-XXX".
   - Defina um `status_validacao_final` claro e informativo.

**Formato de Saída OBRIGATÓRIO (Retorne APENAS o objeto JSON):**
{{{{  // Chave dupla aqui para o Python não tentar formatar o JSON de exemplo
  "status_consulta_cep": "Um status como 'Encontrado' ou 'CEP da Planilha Inválido'", // Exemplo de valor, não um placeholder para Python
  "logradouro_final": "Rua Exemplo Corrigido",
  "numero_final": "123",
  "complemento_final": "Bloco A, Apto 101",
  "bairro_final": "Bairro Exemplo Corrigido",
  "cidade_final": "Cidade Exemplo Corrigida",
  "uf_final": "SP",
  "cep_final": "12345-678",
  "endereco_completo_final": "Rua Exemplo Corrigido, 123, Bloco A, Apto 101, Bairro Exemplo Corrigido - Cidade Exemplo Corrigida/SP, CEP: 12345-678",
  "status_validacao_final": "EXEMPLO_DE_STATUS_FINAL", // Veja exemplos abaixo
  "observacao_validacao": "Exemplo de observação sobre as correções feitas."
}}}} // Chave dupla aqui

**Exemplos de `status_validacao_final` que VOCÊ DEVE GERAR no JSON de saída:**
- `VALIDO_CONFIRMADO_PELO_CEP`
- `CORRIGIDO_E_VALIDADO_COM_CEP`
- `CORRIGIDO_COM_IA_DIVERGENCIA_CEP`
- `CORRIGIDO_COM_IA_CEP_INVALIDO`
- `CORRIGIDO_COM_IA_SEM_CEP_PLANILHA`
- `SUGESTAO_CORRECAO_REVISAR`
- `ERRO_NAO_CORRIGIVEL_REVISAR`
- `CEP_NAO_FORNECIDO_VALIDACAO_LIMITADA`

**Importante:**
- Seu foco é retornar o endereço MAIS PRECISO E COMPLETO POSSÍVEL.
- Seja meticuloso com a padronização de abreviações.
- Se o número do endereço original for "SN", "S/N", "Sem Número", represente-o como uma string vazia ou `null` no campo `numero_final`, e omita-o do `endereco_completo_final` se for o caso.
- **Retorne APENAS o objeto JSON.**
"""

print("INFO: Prompt do Gemini (Multi-Fase) CORRIGIDO e definido.")

INFO: Prompt do Gemini (Multi-Fase) CORRIGIDO e definido.


In [46]:
# gemini_handler.py
import google.generativeai as genai
import json
import traceback

# GEMINI_API_KEY, GEMINI_MODEL_ID, GEMINI_MULTI_FASE_VALIDATION_PROMPT
# e nomes de colunas são definidos nas Células 2 e 4.

try:
    if GEMINI_API_KEY:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel(
            GEMINI_MODEL_ID,
            generation_config={"response_mime_type": "application/json", "temperature": 0.25}, # Temp um pouco maior para tarefas complexas
            safety_settings=[
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            ]
        )
        print(f"INFO: Modelo Gemini '{GEMINI_MODEL_ID}' inicializado no gemini_handler (multi-fase).")
    else:
        model = None
        print("ERRO: GEMINI_API_KEY não disponível. Gemini Handler não pode inicializar o modelo.")
except Exception as e:
    print(f"ERRO ao inicializar o modelo Gemini no gemini_handler (multi-fase): {e}")
    model = None
    traceback.print_exc()

def validate_address_with_gemini_multifase(
    address_planilha: dict,
    address_viacep: dict # Pode ser None ou ter uma estrutura de erro do ViaCEP
    ) -> dict:
    """
    Envia o endereço da planilha e o do ViaCEP para o Gemini para validação multi-fase.
    """
    if not model:
        return {"status_validacao_final": "ERRO_MODELO_NAO_INICIALIZADO", "observacao_validacao": "Modelo Gemini não carregado."}

    # Prepara os dados da planilha para o prompt
    logradouro_p = address_planilha.get(COL_LOGRADOURO_ORIGINAL, "")
    numero_p = address_planilha.get(COL_NUMERO_ORIGINAL, "")
    complemento_p = address_planilha.get(COL_COMPLEMENTO_ORIGINAL, "")
    bairro_p = address_planilha.get(COL_BAIRRO_ORIGINAL, "")
    cidade_p = address_planilha.get(COL_CIDADE_ORIGINAL, "")
    uf_p = address_planilha.get(COL_UF_ORIGINAL, "")
    cep_p_original = address_planilha.get(COL_CEP_ORIGINAL, "")
    cep_p_formatado = clean_cep(str(cep_p_original)) # Usa a função da Célula 3
    if len(cep_p_formatado) == 8:
        cep_p_formatado = f"{cep_p_formatado[:5]}-{cep_p_formatado[5:]}"
    else:
        cep_p_formatado = cep_p_original # Mantém original se não for formatável para 8 dígitos

    endereco_planilha_str = f"Logradouro: {logradouro_p}, Número: {numero_p}, Complemento: {complemento_p}, Bairro: {bairro_p}, Cidade: {cidade_p}, UF: {uf_p}, CEP: {cep_p_formatado}"

    # Prepara os dados do ViaCEP para o prompt
    status_vc = "N/A"
    logradouro_vc = "N/A"
    bairro_vc = "N/A"
    cidade_vc = "N/A"
    uf_vc = "N/A"

    if address_viacep:
        status_vc = address_viacep.get("status_consulta_cep", "Erro desconhecido na consulta CEP")
        if not address_viacep.get("erro"):
            logradouro_vc = address_viacep.get("logradouro", "")
            bairro_vc = address_viacep.get("bairro", "")
            cidade_vc = address_viacep.get("localidade", "") # ViaCEP usa 'localidade'
            uf_vc = address_viacep.get("uf", "")
    else: # Caso address_viacep seja None
        status_vc = "CEP não fornecido ou não consultado"


    response_text_for_error = "N/A (antes da chamada à API Gemini)"
    try:
        prompt_filled = GEMINI_MULTI_FASE_VALIDATION_PROMPT.format(
            endereco_planilha_str=endereco_planilha_str,
            logradouro_planilha=logradouro_p,
            numero_planilha=numero_p,
            complemento_planilha=complemento_p,
            bairro_planilha=bairro_p,
            cidade_planilha=cidade_p,
            uf_planilha=uf_p,
            cep_planilha=cep_p_formatado,
            status_consulta_cep=status_vc,
            logradouro_cep=logradouro_vc,
            bairro_cep=bairro_vc,
            cidade_cep=cidade_vc,
            uf_cep=uf_vc
        )

        print(f"DEBUG (Gemini): Enviando prompt multi-fase (primeiros 400 chars):\n{prompt_filled[:400]}...")
        response = model.generate_content(prompt_filled)
        response_text_for_error = response.text

        cleaned_response_text = response.text.strip()
        if cleaned_response_text.startswith("```json"): cleaned_response_text = cleaned_response_text[7:]
        if cleaned_response_text.endswith("```"): cleaned_response_text = cleaned_response_text[:-3]
        cleaned_response_text = cleaned_response_text.strip()

        if not cleaned_response_text:
            print("ALERTA (Gemini): Resposta (após limpeza) está vazia.")
            return {"status_validacao_final": "ERRO_GEMINI_RESPOSTA_VAZIA", "observacao_validacao": "Gemini retornou resposta vazia."}

        print(f"DEBUG (Gemini): Resposta JSON (limpa) (primeiros 300 chars): {cleaned_response_text[:300]}...")
        gemini_output = json.loads(cleaned_response_text)
        return gemini_output

    except json.JSONDecodeError as json_err:
        print(f"ERRO CRÍTICO (Gemini): Falha ao decodificar JSON: {json_err}")
        print(f"Resposta bruta do Gemini que causou o erro: '{response_text_for_error}'")
        return {"status_validacao_final": "ERRO_PARSE_JSON_GEMINI", "observacao_validacao": f"Falha JSON: {json_err}. Resposta: {response_text_for_error[:200]}"}
    except Exception as e:
        print(f"ERRO CRÍTICO (Gemini): Chamada à API ou processamento: {e}")
        traceback.print_exc()
        error_details = f"Exceção: {str(e)}."
        if hasattr(response, 'prompt_feedback') and response.prompt_feedback and response.prompt_feedback.block_reason:
            error_details += f" Motivo do bloqueio: {response.prompt_feedback.block_reason}."
        return {"status_validacao_final": "ERRO_API_GEMINI_GERAL", "observacao_validacao": error_details}

print("INFO: Handler do Gemini (Multi-Fase) definido.")

INFO: Modelo Gemini 'gemini-1.5-flash-latest' inicializado no gemini_handler (multi-fase).
INFO: Handler do Gemini (Multi-Fase) definido.


In [47]:
# geocoding_handler.py
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
import traceback
import urllib.parse # Para criar URLs de busca do Google Maps

# Nomes de colunas COL_..._FINAL são da Célula 2 (config.py)

geolocator = Nominatim(user_agent="delphos_address_validator_colab_refactored/1.0", timeout=20)
geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, error_wait_seconds=10.0, max_retries=2, return_value_on_exception=None)

def get_geolocation(address_str: str) -> tuple:
    if not address_str or not isinstance(address_str, str) or len(address_str.strip()) < 10: # Aumentar um pouco o mínimo
        print(f"ALERTA (Geocoding): Endereço para geocodificação inválido ou muito curto: '{address_str}'")
        return None, None
    try:
        print(f"INFO (Geocoding): Geocodificando: '{address_str}'")
        location = geocode_with_delay(address_str)

        if location:
            print(f"INFO (Geocoding): Localização encontrada: Lat {location.latitude}, Lon {location.longitude}")
            return location.latitude, location.longitude
        else:
            print(f"ALERTA (Geocoding): Localização não encontrada para '{address_str}'.")
            return None, None
    except Exception as e:
        print(f"ERRO (Geocoding): Durante a geocodificação para '{address_str}': {e}")
        traceback.print_exc()
        return None, None

def format_address_for_geocoding_from_final_data(address_final_dict: dict) -> str:
    """Usa os campos FINAIS (corrigidos pelo Gemini) para formatar o endereço para geolocalização."""
    logradouro = address_final_dict.get(COL_LOGRADOURO_FINAL, "")
    numero = address_final_dict.get(COL_NUMERO_FINAL, "")
    bairro = address_final_dict.get(COL_BAIRRO_FINAL, "")
    cidade = address_final_dict.get(COL_CIDADE_FINAL, "")
    uf = address_final_dict.get(COL_UF_FINAL, "")
    cep = address_final_dict.get(COL_CEP_FINAL, "") # Usar o CEP final corrigido

    components = [
        str(logradouro or "").strip(), str(numero or "").strip(),
        str(bairro or "").strip(), str(cidade or "").strip(),
        str(uf or "").strip(), str(cep or "").strip()
    ]
    full_address = ", ".join(filter(None, components))
    return full_address

def generate_Maps_link(lat: float = None, lon: float = None, address_query: str = None) -> str:
    """Gera um link do Google Maps a partir de coordenadas ou de uma string de endereço."""
    base_url = "https://www.google.com/maps?"
    if lat is not None and lon is not None:
        return f"{base_url}q={lat},{lon}"
    elif address_query and address_query.strip():
        # Codifica a string de endereço para ser usada na URL
        encoded_query = urllib.parse.quote_plus(address_query)
        return f"https://www.google.com/maps/search/?api=1&query={encoded_query}"
    return ""

print("INFO: Handler de Geolocalização (com link Maps) definido.")

INFO: Handler de Geolocalização (com link Maps) definido.


In [48]:
# main.py (simplificado)
import pandas as pd
import json
import time
import traceback

# Funções e constantes das células anteriores estão no escopo:
# Célula 2: config (COL_..., ALL_RESULT_COLUMNS, etc.)
# Célula 3: prompts (GEMINI_ADDRESS_VALIDATION_PROMPT)
# Célula 4: gemini_handler (validate_address_with_gemini)
# Célula 5: geocoding_handler (get_geolocation, format_address_for_geocoding)
# Célula 6: google_sheets_handler (read_addresses_from_sheet, update_sheet_with_results)

def main_process_addresses(df_input: pd.DataFrame) -> pd.DataFrame:
    """
    Orquestra o processo de validação e geolocalização de endereços.
    """
    results_accumulator = [] # Lista para acumular os dicionários de resultado de cada linha

    if 'original_row_index' not in df_input.columns:
        print("ALERTA: Coluna 'original_row_index' não encontrada no DataFrame de entrada. Adicionando-a, assumindo que os dados começam na linha 2 da planilha após o cabeçalho.")
        # Se read_addresses_from_sheet não a adicionou, pode indicar problema anterior.
        # Mas, para robustez, vamos adicioná-la aqui se ausente.
        df_input['original_row_index'] = df_input.index + 2

    for index, row_series in df_input.iterrows():
        # Converte a Series da linha para um dicionário para fácil acesso por .get()
        row_dict = row_series.to_dict()

        # Prepara os dados do endereço original para enviar ao Gemini
        address_data_for_gemini = {
            COL_LOGRADOURO_ORIGINAL: row_dict.get(COL_LOGRADOURO_ORIGINAL),
            COL_NUMERO_ORIGINAL: row_dict.get(COL_NUMERO_ORIGINAL),
            COL_COMPLEMENTO_ORIGINAL: row_dict.get(COL_COMPLEMENTO_ORIGINAL),
            COL_BAIRRO_ORIGINAL: row_dict.get(COL_BAIRRO_ORIGINAL),
            COL_CIDADE_ORIGINAL: row_dict.get(COL_CIDADE_ORIGINAL),
            COL_UF_ORIGINAL: row_dict.get(COL_UF_ORIGINAL),
            COL_CEP_ORIGINAL: row_dict.get(COL_CEP_ORIGINAL)
        }

        current_row_index_for_log = row_dict.get('original_row_index', index + 2) # Usa o índice original se disponível
        print(f"\n--- Processando linha {current_row_index_for_log}: {row_dict.get(COL_LOGRADOURO_ORIGINAL, 'Endereço não fornecido')} ---")

        gemini_response_dict = validate_address_with_gemini(address_data_for_gemini)

        # Começa com os dados da linha original para manter quaisquer outras colunas que possam existir
        # e para ter os dados originais disponíveis para referência.
        output_row_dict = row_dict.copy()

        output_row_dict[COL_JSON_GEMINI] = json.dumps(gemini_response_dict, ensure_ascii=False) if gemini_response_dict else "{}"
        output_row_dict[COL_STATUS_VALIDACAO] = gemini_response_dict.get("status", "erro_desconhecido") if gemini_response_dict else "erro_gemini_sem_resposta"
        output_row_dict[COL_OBSERVACAO_GEMINI] = gemini_response_dict.get("observacao", "") if gemini_response_dict else "Nenhuma observação do Gemini."

        if gemini_response_dict and gemini_response_dict.get("status") in ["corrigido", "valido", "sugestao_correcao"]:
            # Preenche as colunas corrigidas/sugeridas
            output_row_dict[COL_LOGRADOURO_CORRIGIDO] = gemini_response_dict.get("logradouro_corrigido") or gemini_response_dict.get("logradouro_sugerido")
            output_row_dict[COL_NUMERO_CORRIGIDO] = gemini_response_dict.get("numero_corrigido") or gemini_response_dict.get("numero_sugerido")
            output_row_dict[COL_COMPLEMENTO_CORRIGIDO] = gemini_response_dict.get("complemento_corrigido") or gemini_response_dict.get("complemento_sugerido")
            output_row_dict[COL_BAIRRO_CORRIGIDO] = gemini_response_dict.get("bairro_corrigido") or gemini_response_dict.get("bairro_sugerido")
            output_row_dict[COL_CIDADE_CORRIGIDA] = gemini_response_dict.get("cidade_corrigida") or gemini_response_dict.get("cidade_sugerida")
            output_row_dict[COL_UF_CORRIGIDA] = gemini_response_dict.get("uf_corrigida") or gemini_response_dict.get("uf_sugerida")
            output_row_dict[COL_CEP_CORRIGIDO] = gemini_response_dict.get("cep_corrigido") or gemini_response_dict.get("cep_sugerido")

            # Prepara e executa a geolocalização
            address_str_for_geo = format_address_for_geocoding(output_row_dict) # Passa o dicionário com os campos corrigidos
            output_row_dict[COL_ENDERECO_COMPLETO_PARA_GEO] = address_str_for_geo

            if address_str_for_geo and address_str_for_geo.strip(): # Só geolocaliza se tiver um endereço formatado
                lat, lon = get_geolocation(address_str_for_geo)
                output_row_dict[COL_LATITUDE] = lat
                output_row_dict[COL_LONGITUDE] = lon
            else:
                print(f"ALERTA: Endereço formatado para geocodificação resultou vazio para linha {current_row_index_for_log}. Geolocalização pulada.")
                output_row_dict[COL_LATITUDE] = None
                output_row_dict[COL_LONGITUDE] = None
        else:
            # Se o status do Gemini não for de sucesso ou sugestão, não tenta geolocalizar
            status_gemini = output_row_dict[COL_STATUS_VALIDACAO]
            print(f"ALERTA: Status da validação Gemini ('{status_gemini}') não permite geocodificação para linha {current_row_index_for_log}.")
            output_row_dict[COL_LATITUDE] = None
            output_row_dict[COL_LONGITUDE] = None
            output_row_dict[COL_ENDERECO_COMPLETO_PARA_GEO] = "" # Limpa, pois não houve correção válida

        results_accumulator.append(output_row_dict)
        print(f"INFO: Pausando por 1 segundo antes da próxima linha...")
        time.sleep(1) # Pausa para não sobrecarregar APIs

    return pd.DataFrame(results_accumulator)


# --- Bloco principal para execução no Colab ---
if __name__ == "__main__": # Garante que só rode quando executado diretamente
    print(f"INFO: Iniciando processo de validação de endereços (Planilha: '{SHEET_NAME_TO_PROCESS}')...")

    if not GEMINI_API_KEY: # Verifica se a API Key foi carregada (definida na Célula 2)
        print("ERRO CRÍTICO: GEMINI_API_KEY não está configurada. Verifique a Célula 2 (config.py) e os Secrets do Colab.")
    else:
        # 1. Ler endereços da planilha
        df_input_addresses = read_addresses_from_sheet(SHEET_NAME_TO_PROCESS)

        if not df_input_addresses.empty:
            print(f"INFO: Foram lidos {len(df_input_addresses)} endereços da planilha.")

            # 2. Processar os endereços (validação com Gemini + geolocalização)
            df_output_processed = main_process_addresses(df_input_addresses.copy()) # Usa uma cópia para evitar modificar o original no loop

            # 3. Atualizar a planilha com os resultados
            if not df_output_processed.empty:
                print(f"\nINFO: Processamento concluído. {len(df_output_processed)} endereços foram processados. Atualizando a planilha...")
                update_sheet_with_results(SHEET_NAME_TO_PROCESS, df_output_processed)
                print("INFO: Planilha atualizada com os resultados da validação e geolocalização.")
            else:
                print("ALERTA: Nenhum endereço foi processado ou o DataFrame de resultados processados está vazio. Nenhuma atualização será feita na planilha.")
        else:
            print(f"ALERTA: Nenhum endereço foi encontrado na planilha '{SHEET_NAME_TO_PROCESS}' ou ocorreu um erro ao tentar ler os dados.")

    print("INFO: Processo de validação de endereços finalizado.")

INFO: Iniciando processo de validação de endereços (Planilha: 'enderecos_com_problema')...
INFO: Cliente gspread autenticado com sucesso.
INFO: Abrindo planilha ID: 1zRX--zEYZTaIJIuX6oQObw2V9KGqpDgS242jxehPoL8
INFO: Planilha 'Enderecos' aberta.
INFO: Acessando aba: enderecos_com_problema
INFO: Aba 'enderecos_com_problema' acessada. Lendo registros...
DEBUG: Cabeçalhos esperados para leitura: ['Logradouro Original', 'Número Original', 'Complemento Original', 'Bairro Original', 'Cidade Original', 'UF Original', 'CEP Original']
DEBUG: Cabeçalhos reais na planilha (lidos para validação): ['Logradouro Original', 'Número Original', 'Complemento Original', 'Bairro Original', 'Cidade Original', 'UF Original', 'CEP Original']
INFO: 6 linhas lidas da planilha com get_all_records.
INFO: DataFrame de entrada criado com sucesso com 6 linhas.
INFO: Foram lidos 6 endereços da planilha.

--- Processando linha 2: R LOMAS VALT ---
DEBUG: Enviando prompt para Gemini (primeiros 300 chars): 
Você é um assi

In [49]:
# google_sheets_handler.py
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import traceback

# Constantes e nomes de colunas (COL_..., ALL_RESULT_COLUMNS, GOOGLE_SHEET_ID, etc.)
# são definidos na Célula 2 (config.py)

SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive.file'
]

def get_google_sheets_client():
    try:
        creds = Credentials.from_service_account_file(GOOGLE_SHEET_CREDENTIALS_FILE, scopes=SCOPES)
        client = gspread.authorize(creds)
        print("INFO: Cliente gspread autenticado com sucesso.")
        return client
    except Exception as e:
        print(f"ERRO CRÍTICO ao autenticar cliente gspread: {e}")
        traceback.print_exc()
        return None

def read_addresses_from_sheet(sheet_name: str, client=None) -> pd.DataFrame:
    if client is None:
        client = get_google_sheets_client()
        if client is None:
             print("ERRO: Não foi possível obter o cliente gspread. Abortando leitura.")
             return pd.DataFrame()
    try:
        print(f"INFO: Abrindo planilha ID: {GOOGLE_SHEET_ID}")
        spreadsheet = client.open_by_key(GOOGLE_SHEET_ID)
        print(f"INFO: Planilha '{spreadsheet.title}' aberta.")
        print(f"INFO: Acessando aba: {sheet_name}")
        worksheet = spreadsheet.worksheet(sheet_name)
        print(f"INFO: Aba '{worksheet.title}' acessada. Lendo registros...")

        expected_input_cols = [
            COL_LOGRADOURO_ORIGINAL, COL_NUMERO_ORIGINAL, COL_COMPLEMENTO_ORIGINAL,
            COL_BAIRRO_ORIGINAL, COL_CIDADE_ORIGINAL, COL_UF_ORIGINAL, COL_CEP_ORIGINAL
        ]
        print(f"DEBUG: Cabeçalhos esperados para leitura: {expected_input_cols}")

        try:
            actual_headers = worksheet.row_values(1)
            print(f"DEBUG: Cabeçalhos reais na planilha (lidos para validação): {actual_headers}")
            if not all(expected_header in actual_headers for expected_header in expected_input_cols):
                print("ALERTA DE LEITURA: Discrepância detectada entre cabeçalhos esperados e os cabeçalhos reais na planilha.")
        except Exception as he:
            print(f"ALERTA DE LEITURA: Não foi possível ler/comparar os cabeçalhos reais da planilha: {he}")

        data = worksheet.get_all_records(expected_headers=expected_input_cols, head=1, default_blank="") # head=1 para garantir que use a primeira linha como cabeçalho, default_blank para tratar células vazias
        print(f"INFO: {len(data)} linhas lidas da planilha com get_all_records.")

        if not data:
            print("ALERTA DE LEITURA: Nenhum dado retornado por get_all_records.")
            return pd.DataFrame()

        df = pd.DataFrame(data)
        if df.empty:
            print("ALERTA DE LEITURA: DataFrame está vazio após a conversão.")
            return pd.DataFrame()

        df['original_row_index'] = df.index + 2
        print(f"INFO: DataFrame de entrada criado com sucesso com {len(df)} linhas.")
        return df
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"ERRO CRÍTICO DE LEITURA: Planilha com ID '{GOOGLE_SHEET_ID}' não encontrada.")
        return pd.DataFrame()
    except gspread.exceptions.WorksheetNotFound:
        print(f"ERRO CRÍTICO DE LEITURA: Aba '{sheet_name}' não encontrada na planilha.")
        return pd.DataFrame()
    except Exception as e:
        print(f"ERRO CRÍTICO geral ao ler dados da planilha: {e}")
        traceback.print_exc()
        return pd.DataFrame()


def update_sheet_with_results(sheet_name: str, df_results: pd.DataFrame, client=None):
    if df_results.empty:
        print("INFO (Update): DataFrame de resultados está vazio. Nenhuma atualização será feita.")
        return

    if client is None:
        client = get_google_sheets_client()
        if client is None:
            print("ERRO (Update): Não foi possível obter cliente gspread. Abortando atualização.")
            return
    try:
        print(f"INFO (Update): Atualizando planilha ID: {GOOGLE_SHEET_ID}, Aba: {sheet_name}")
        spreadsheet = client.open_by_key(GOOGLE_SHEET_ID)
        worksheet = spreadsheet.worksheet(sheet_name)
        print(f"INFO (Update): Aba '{worksheet.title}' acessada para atualização.")

        for col_name_cfg in ALL_RESULT_COLUMNS:
            if col_name_cfg not in df_results.columns:
                df_results[col_name_cfg] = ''

        current_sheet_headers = worksheet.row_values(1)
        print(f"DEBUG (Update): Cabeçalhos atuais NA PLANILHA antes de qualquer modificação: {current_sheet_headers}")

        existing_header_to_index_map = {header: i + 1 for i, header in enumerate(current_sheet_headers) if header}

        final_col_name_to_sheet_index_map = {}
        header_cells_to_add_or_update_on_sheet = []

        next_available_col_idx_for_new_header = len(current_sheet_headers) + 1
        if all(s == '' for s in current_sheet_headers):
             next_available_col_idx_for_new_header = 1

        print(f"DEBUG (Update): Colunas que o script VAI TENTAR ESCREVER (de ALL_RESULT_COLUMNS): {ALL_RESULT_COLUMNS}")

        for col_name_from_config in ALL_RESULT_COLUMNS:
            if col_name_from_config in existing_header_to_index_map:
                final_col_name_to_sheet_index_map[col_name_from_config] = existing_header_to_index_map[col_name_from_config]
                print(f"DEBUG (Update): Coluna '{col_name_from_config}' JÁ EXISTE na planilha no índice {existing_header_to_index_map[col_name_from_config]}.")
            else:
                final_col_name_to_sheet_index_map[col_name_from_config] = next_available_col_idx_for_new_header
                header_cells_to_add_or_update_on_sheet.append(gspread.Cell(row=1, col=next_available_col_idx_for_new_header, value=col_name_from_config))
                print(f"DEBUG (Update): Cabeçalho da NOVA coluna '{col_name_from_config}' será adicionado na planilha no índice {next_available_col_idx_for_new_header}.")
                next_available_col_idx_for_new_header += 1

        if header_cells_to_add_or_update_on_sheet:
            worksheet.update_cells(header_cells_to_add_or_update_on_sheet, value_input_option='USER_ENTERED')
            print(f"INFO (Update): {len(header_cells_to_add_or_update_on_sheet)} novos cabeçalhos foram adicionados/atualizados na planilha.")
            # Re-ler cabeçalhos e recriar mapa se novos foram adicionados pode ser mais seguro
            current_sheet_headers = worksheet.row_values(1)
            final_col_name_to_sheet_index_map = {header: i + 1 for i, header in enumerate(current_sheet_headers)}
            print(f"DEBUG (Update): Mapeamento de colunas REFEITO após adição de cabeçalhos: {final_col_name_to_sheet_index_map}")

        else:
            print("DEBUG (Update): Nenhum novo cabeçalho precisou ser adicionado.")

        print(f"DEBUG (Update): Mapeamento FINAL de NomeDeColunaParaÍndiceNaPlanilha antes de escrever dados: {final_col_name_to_sheet_index_map}")

        cells_to_update_data_on_sheet = []
        if 'original_row_index' not in df_results.columns:
            print("ERRO CRÍTICO (Update): Coluna 'original_row_index' não encontrada no DataFrame de resultados.")
            return

        for df_idx, df_row_series in df_results.iterrows():
            sheet_row_number = int(df_row_series['original_row_index'])
            print(f"DEBUG (Update): Preparando dados para linha {sheet_row_number} da planilha...")
            for col_name_to_write in ALL_RESULT_COLUMNS:
                sheet_col_index = final_col_name_to_sheet_index_map.get(col_name_to_write)

                if sheet_col_index is not None:
                    value_from_df = df_row_series.get(col_name_to_write)
                    if pd.isna(value_from_df): value_from_df = ''
                    else: value_from_df = str(value_from_df)

                    print(f"  -> Planilha: Linha {sheet_row_number}, Coluna {sheet_col_index} ('{col_name_to_write}'). Valor: '{value_from_df[:60]}...'")
                    cells_to_update_data_on_sheet.append(gspread.Cell(row=sheet_row_number, col=sheet_col_index, value=value_from_df))
                else:
                    print(f"ALERTA (Update): Coluna '{col_name_to_write}' não foi mapeada para um índice na planilha. Dados não serão escritos para esta coluna na linha {sheet_row_number}.")

        if cells_to_update_data_on_sheet:
            print(f"INFO (Update): Enviando {len(cells_to_update_data_on_sheet)} células de dados para atualização...")
            worksheet.update_cells(cells_to_update_data_on_sheet, value_input_option='USER_ENTERED')
            print(f"INFO (Update): Células de dados da planilha '{sheet_name}' atualizadas com sucesso.")
        else:
            print("INFO (Update): Nenhuma célula de dados foi preparada para atualização.")

    except Exception as e:
        print(f"ERRO CRÍTICO ao atualizar planilha: {e}")
        traceback.print_exc()

print("INFO: Handler do Google Sheets (com depuração para atualização) definido.")

INFO: Handler do Google Sheets (com depuração para atualização) definido.


In [52]:
# main.py (Refeito para novo fluxo com ViaCEP e Gemini Multi-Fase)
import pandas as pd
import json
import time
import traceback

# Funções e constantes das células anteriores estão no escopo:
# Célula 2: config (COL_..., ALL_RESULT_COLUMNS, etc.)
# Célula 3: cep_handler (get_address_from_viacep, clean_cep)
# Célula 4: prompts (GEMINI_MULTI_FASE_VALIDATION_PROMPT)
# Célula 5: gemini_handler (validate_address_with_gemini_multifase)
# Célula 6: geocoding_handler (get_geolocation, format_address_for_geocoding_from_final_data, generate_Maps_link)
# Célula 7: google_sheets_handler (read_addresses_from_sheet, update_sheet_with_results)

def main_process_new_flow(df_input: pd.DataFrame) -> pd.DataFrame:
    results_accumulator = []

    if 'original_row_index' not in df_input.columns:
        df_input['original_row_index'] = df_input.index + 2

    for index, row_series in df_input.iterrows():
        row_dict_original = row_series.to_dict()
        output_row_data = row_dict_original.copy() # Começa com todos os dados originais

        current_row_idx_log = output_row_data.get('original_row_index', index + 2)
        print(f"\n--- Iniciando Processamento para Linha da Planilha: {current_row_idx_log} ---")
        print(f"Dados Originais: { {k:v for k,v in row_dict_original.items() if k.endswith('Original')} }")


        # Fase 2: Buscar endereço pelo CEP (ViaCEP)
        cep_original_planilha = str(row_dict_original.get(COL_CEP_ORIGINAL, "")).strip()
        address_from_viacep = None
        status_consulta_cep_para_gemini = "CEP não fornecido na planilha"

        if cep_original_planilha:
            address_from_viacep = get_address_from_viacep(cep_original_planilha)
            status_consulta_cep_para_gemini = address_from_viacep.get("status_consulta_cep", "Erro na consulta ViaCEP")
            if not address_from_viacep.get("erro"):
                output_row_data[COL_CEP_CONSULTADO_LOGRADOURO] = address_from_viacep.get("logradouro")
                output_row_data[COL_CEP_CONSULTADO_BAIRRO] = address_from_viacep.get("bairro")
                output_row_data[COL_CEP_CONSULTADO_CIDADE] = address_from_viacep.get("localidade")
                output_row_data[COL_CEP_CONSULTADO_UF] = address_from_viacep.get("uf")
            output_row_data[COL_STATUS_CONSULTA_CEP] = status_consulta_cep_para_gemini
        else:
            output_row_data[COL_STATUS_CONSULTA_CEP] = status_consulta_cep_para_gemini
            print("INFO: CEP original não fornecido na planilha. Pulando consulta ViaCEP.")

        # Prepara o dicionário do endereço original da planilha para o Gemini
        address_planilha_for_gemini = {
            COL_LOGRADOURO_ORIGINAL: row_dict_original.get(COL_LOGRADOURO_ORIGINAL),
            COL_NUMERO_ORIGINAL: row_dict_original.get(COL_NUMERO_ORIGINAL),
            COL_COMPLEMENTO_ORIGINAL: row_dict_original.get(COL_COMPLEMENTO_ORIGINAL),
            COL_BAIRRO_ORIGINAL: row_dict_original.get(COL_BAIRRO_ORIGINAL),
            COL_CIDADE_ORIGINAL: row_dict_original.get(COL_CIDADE_ORIGINAL),
            COL_UF_ORIGINAL: row_dict_original.get(COL_UF_ORIGINAL),
            COL_CEP_ORIGINAL: cep_original_planilha # Passa o CEP original (não o formatado ainda, Gemini fará isso)
        }

        # Fase 3 & 4: Validação com Gemini (que agora recebe ambos os endereços)
        print(f"INFO: Enviando para Gemini para validação multi-fase...")
        gemini_final_response = validate_address_with_gemini_multifase(
            address_planilha_for_gemini,
            address_from_viacep # Passa o resultado completo do ViaCEP
        )

        # Preenche as colunas de resultado do Gemini
        output_row_data[COL_JSON_GEMINI_FINAL] = json.dumps(gemini_final_response, ensure_ascii=False)
        output_row_data[COL_STATUS_VALIDACAO_FINAL] = gemini_final_response.get("status_validacao_final", "ERRO_GEMINI_SEM_STATUS_FINAL")
        output_row_data[COL_OBSERVACAO_VALIDACAO] = gemini_final_response.get("observacao_validacao", "")

        # Se Gemini retornou um status de sucesso ou sugestão, preenche os campos finais
        if gemini_final_response.get("status_validacao_final") and not gemini_final_response.get("status_validacao_final").startswith("ERRO"):
            output_row_data[COL_LOGRADOURO_FINAL] = gemini_final_response.get("logradouro_final")
            output_row_data[COL_NUMERO_FINAL] = gemini_final_response.get("numero_final")
            output_row_data[COL_COMPLEMENTO_FINAL] = gemini_final_response.get("complemento_final")
            output_row_data[COL_BAIRRO_FINAL] = gemini_final_response.get("bairro_final")
            output_row_data[COL_CIDADE_FINAL] = gemini_final_response.get("cidade_final")
            output_row_data[COL_UF_FINAL] = gemini_final_response.get("uf_final")
            output_row_data[COL_CEP_FINAL] = gemini_final_response.get("cep_final")
            output_row_data[COL_ENDERECO_COMPLETO_FINAL] = gemini_final_response.get("endereco_completo_final")

            # Fase 5: Geolocalização
            # Usa o COL_ENDERECO_COMPLETO_FINAL para geolocalização, pois é o mais completo.
            # Ou, alternativamente, a função format_address_for_geocoding_from_final_data
            # que reconstrói a partir dos campos finais individuais.

            endereco_para_geo_str = output_row_data[COL_ENDERECO_COMPLETO_FINAL]
            if not endereco_para_geo_str or not endereco_para_geo_str.strip():
                 # Fallback se o endereço completo final estiver vazio, tenta construir dos campos finais
                 endereco_para_geo_str = format_address_for_geocoding_from_final_data(output_row_data)

            output_row_data[COL_ENDERECO_COMPLETO_PARA_GEO] = endereco_para_geo_str # Guarda o que foi usado para geolocalizar

            if endereco_para_geo_str and endereco_para_geo_str.strip():
                lat, lon = get_geolocation(endereco_para_geo_str)
                output_row_data[COL_LATITUDE] = lat
                output_row_data[COL_LONGITUDE] = lon
                output_row_data[COL_LINK_Maps] = generate_Maps_link(lat=lat, lon=lon, address_query=endereco_para_geo_str)
            else:
                print(f"ALERTA (Main): Endereço para geolocalização está vazio para linha {current_row_idx_log}.")
                output_row_data[COL_LATITUDE] = None; output_row_data[COL_LONGITUDE] = None; output_row_data[COL_LINK_Maps] = ""
        else:
            status_final_gemini = output_row_data[COL_STATUS_VALIDACAO_FINAL]
            print(f"ALERTA (Main): Status final da validação Gemini ('{status_final_gemini}') indica erro ou não permite geocodificação para linha {current_row_idx_log}.")
            # Limpa campos de geolocalização e endereço completo se a validação falhou
            output_row_data[COL_LATITUDE] = None; output_row_data[COL_LONGITUDE] = None; output_row_data[COL_LINK_Maps] = ""
            output_row_data[COL_ENDERECO_COMPLETO_FINAL] = ""
            output_row_data[COL_ENDERECO_COMPLETO_PARA_GEO] = ""


        results_accumulator.append(output_row_data)
        print(f"INFO (Main): Pausando por 1 segundo...")
        time.sleep(1)

    return pd.DataFrame(results_accumulator)


# --- Bloco Principal para Execução ---
if __name__ == "__main__":
    print(f"INFO: Iniciando NOVO FLUXO de validação de endereços (Planilha: '{SHEET_NAME_TO_PROCESS}')...")

    if not GEMINI_API_KEY:
        print("ERRO CRÍTICO: GEMINI_API_KEY não está configurada. Verifique a Célula 2 (config.py) e os Secrets do Colab.")
    else:
        df_input = read_addresses_from_sheet(SHEET_NAME_TO_PROCESS)

        if not df_input.empty:
            print(f"INFO: Foram lidos {len(df_input)} endereços da planilha.")

            # Garante que o DataFrame de entrada tenha todas as colunas de resultado (preenchidas com None)
            # para que o DataFrame final df_processed tenha a estrutura correta mesmo que algumas linhas
            # não sejam totalmente processadas ou o Gemini não retorne todos os campos.
            for col_res in ALL_RESULT_COLUMNS:
                if col_res not in df_input.columns:
                    df_input[col_res] = None

            df_processed_output = main_process_new_flow(df_input.copy())

            if not df_processed_output.empty:
                print(f"\nINFO: Processamento (novo fluxo) concluído. {len(df_processed_output)} endereços processados. Atualizando planilha...")
                update_sheet_with_results(SHEET_NAME_TO_PROCESS, df_processed_output)
                print("INFO: Planilha atualizada com os resultados do novo fluxo.")
            else:
                print("ALERTA: Nenhum endereço foi processado no novo fluxo ou o DataFrame de resultados está vazio.")
        else:
            print(f"ALERTA: Nenhum endereço encontrado na planilha '{SHEET_NAME_TO_PROCESS}' ou ocorreu um erro ao ler os dados.")

    print("INFO: Processo (novo fluxo) finalizado.")

INFO: Iniciando NOVO FLUXO de validação de endereços (Planilha: 'enderecos_com_problema')...
INFO: Cliente gspread autenticado com sucesso.
INFO: Abrindo planilha ID: 1zRX--zEYZTaIJIuX6oQObw2V9KGqpDgS242jxehPoL8
INFO: Planilha 'Enderecos' aberta.
INFO: Acessando aba: enderecos_com_problema
INFO: Aba 'enderecos_com_problema' acessada. Lendo registros...
DEBUG: Cabeçalhos esperados para leitura: ['Logradouro Original', 'Número Original', 'Complemento Original', 'Bairro Original', 'Cidade Original', 'UF Original', 'CEP Original']
DEBUG: Cabeçalhos reais na planilha (lidos para validação): ['Logradouro Original', 'Número Original', 'Complemento Original', 'Bairro Original', 'Cidade Original', 'UF Original', 'CEP Original', 'Logradouro (ViaCEP)', 'Bairro (ViaCEP)', 'Cidade (ViaCEP)', 'UF (ViaCEP)', 'Status Consulta CEP', 'Logradouro Final Corrigido', 'Número Final Corrigido', 'Complemento Final Corrigido', 'Bairro Final Corrigido', 'Cidade Final Corrigida', 'UF Final Corrigida', 'CEP Final 